Goal: *Plot cumulative slip at three different time steps in a single figure.*

What this notebook does: *Loads data and lets the user choose the variable and three time steps to produce a single figure. The plotting is non-interactive and saves screenshots in the format of `<variable_name>-t(<time_step_a>, <time_step_b>, <time_step_c>).png`

In [ ]:
import numpy as np
import pyvista as pv
from tqdm import tqdm

from kale import Engine, save_movie, show_ui
from kale.algorithms import (
    contour_banded,
    subdivide_algorithm,
    scalars_operation_algorithm,
)
from kale import helpers
from kale import theme

In [ ]:
# Theming adjustments
pv.global_theme.image_scale = 10

# Load Data

In [ ]:
MESH_GEOMETRY_FILE_NAME = "2022_11_30_11_52_49_mesh_geometry.vtk"
MESH_VALUES_FILE_NAME = "2022_11_30_11_52_49.hdf"
MESH_Z_SCALE = 0.01

In [ ]:
engine = Engine(MESH_GEOMETRY_FILE_NAME, MESH_VALUES_FILE_NAME, zscale=MESH_Z_SCALE)
engine.keys

# Find Camera Position

This is a helper section to find your preferred camera location

In [ ]:
pl = pv.Plotter()
pl.add_mesh(engine.mesh, color="beige")
pl.show()

In [ ]:
# copy value of this output into next cell after moving scene above
pl.camera_position

In [ ]:
# set the camera position (different for different datasets)
cpos = [
    (246.72654935831832, 56.640050977986135, 19.47264855846906),
    (235.64372937685064, 45.67443514953468, -0.9481135945038834),
    (-0.6591979033815945, -0.45237409343002455, 0.6006794517633125),
]

# Time Steps

List the different time steps to display. The following cell lists available time step range

In [ ]:
# How many time steps available?
engine.max_time_step

In [ ]:
# Set the three time steps
time_step_a = 0  # "pre-event"
time_step_b = engine.max_time_step // 2  # "event"
time_step_c = engine.max_time_step - 10  # "post-event"

time_step_a, time_step_b, time_step_c

In [ ]:
# Chose labels for these three time steps
labels = ["pre-event", "event", "post-event"]

# Choosing data array

The following cell is where you can control which variable is being plotted:

In [ ]:
engine.mesh.active_scalars_name = "cumulative_slip"

# Extracting meshes for each time step

Use a `.copy()` operation to "export" each time step

In [ ]:
engine.time_step = time_step_a
mesh_a = engine.mesh.copy()

engine.time_step = time_step_b
mesh_b = engine.mesh.copy()

engine.time_step = time_step_c
mesh_c = engine.mesh.copy()

# Visualize

This cell will not produce an output, but instead renders off-screen to create screenshots.

In [ ]:
CONTOUR_LEVELS = np.linspace(-10, 10, 21)
N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]
CAMERA_ZOOM = 1.0
TITLE_POSITION = [0.1, 0.62]

boundary = engine.boundary

pl = pv.Plotter(shape=(1, 3), border=False, multi_samples=8, line_smoothing=True)

# Plotter configurations
# pl.enable_ssao(radius=15, bias=0.5)
# pl.enable_anti_aliasing('ssaa')
# pl.enable_shadows()

# Add common features to each subplot
for i in range(3):
    pl.subplot(0, i)
    pl.add_mesh(boundary)
    # pl.add_mesh(pv.Box(mesh.bounds), color="lightgrey", culling="front")
    pl.add_text(
        labels[i],
        position=TITLE_POSITION,
        color="k",
        shadow=False,
        font_size=12,
        viewport=True,
    )
    pl.add_floor("-z", show_edges=True, edge_color="white", color="lightgray")
    helpers.add_bounds(pl)


# TODO: make sure this annotation matches scalar array
# annotations = {
#     0.0: "geometric moment / area (m)",
# }

# Pre-event geometric moment
pl.subplot(0, 0)
contour, edges = contour_banded(
    mesh_a,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars=engine.mesh.active_scalars_name,
)
pl.add_mesh(
    contour,
    cmap="RdYlBu_r",
    clim=CLIM,
    scalars=engine.mesh.active_scalars_name,
    n_colors=N_COLORS,
)
pl.add_mesh(edges)


# Event geometric moment
pl.subplot(0, 1)
# only show scalar bar in this subplot

contour, edges = contour_banded(
    mesh_b,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars=engine.mesh.active_scalars_name,
)
pl.add_mesh(
    contour,
    cmap="RdYlBu_r",
    clim=CLIM,
    scalars=engine.mesh.active_scalars_name,
    n_colors=N_COLORS,
    show_scalar_bar=True,
    scalar_bar_args=dict(title=f"", **theme.SCALAR_BAR_H),
    # annotations=annotations,
)
pl.add_mesh(edges)


# Post-event slip
pl.subplot(0, 2)

contour, edges = contour_banded(
    mesh_c,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars=engine.mesh.active_scalars_name,
)
pl.add_mesh(
    contour,
    cmap="RdYlBu_r",
    clim=CLIM,
    scalars=engine.mesh.active_scalars_name,
    n_colors=N_COLORS,
)
pl.add_mesh(edges)


# Camera controls
pl.link_views()  # links all subplot cameras
pl.camera_position = cpos
# pl.camera.zoom(1.2)

pl.show()

In [ ]:
filename = f"{engine.mesh.active_scalars_name}-t({time_step_a}, {time_step_b}, {time_step_c}).png"
pl.screenshot(filename, scale=3, return_img=False)